# Hoje!

## Exercício!

**Entrega individual de um notebook com esses passos indentificados até dia 15/outubro.**

Com o dataset do projeto, faça o seguinte:
    
1. Crie uma pipeline específica para as seguintes variáveis e realize os tratamentos necessários para transformar essa informação em número ordenado:
    - `NAME_EDUCATION_TYPE`
2. Crie uma pipeline específica para as seguintes variáveis, realizando um One Hot Encoder (leiam a documentação e setem o parâmetro drop conforme fizer sentido para vocês):
    - `FLAG_OWN_CAR`
    - `FLAG_OWN_REALTY`
    - `NAME_INCOME_TYPE`
    - `NAME_FAMILY_STATUS`
    - `OCCUPATION_TYPE`
3. Crie uma pipeline específica para todas as outras variáveis que não são do tipo object. Adicione na pipeline a imputação de missing com a mediana. Com base nos próximos passos, responda a seguinte pergunta:
    - *É necessário realizar o standard scaler nessas variáveis?*
4. Crie uma outra pipeline, que unirá todas as pipelines anteriores. Além de unir os steps passados, essa pipeline deve aplicar uma seleção de features baseada em um modelo de regressão logística. Resposta a seguinte pergunta:
    - *Qual penalidade deve ser usada? Por que?*
5. Após a etapa de seleção de features da pipeline do passo anterior, adicione um modelo de árvore de decisão com `max_depth = 5` e treine o modelo, avaliando as métricas.
6. Faça uma busca em grid nos hiperparâmetros `min_samples_leaf` e `max_depth`, retreine o modelo e avalie as métricas. Responda a seguinte pergunta:
    - *Houve ganho em performance? Se sim, de quanto?* 
7. Com a pipeline treinada, responda a seguinte pergunta:
    - *Quantas e quais features são as que de fato são usadas pelo modelo? Por que?* 

In [33]:
# Geral
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline


# Feat Eng
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from feature_engine.imputation import DropMissingData
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Feat Selection
from sklearn.feature_selection import SelectFromModel

# Modelos
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV


# Métricas e plots
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from plot_metric.functions import BinaryClassification
from mlxtend.plotting import plot_confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# 
import seaborn as sns

np.random.seed(4)


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df_credit_train = pd.read_csv('C:/Users/Avell/Google Drive/Luiz Gabriel - PC/06.Material de Cursos/LetsCode_DS_Curso/06_Machine_Learning_I/projeto/dataset/application_train.csv')

In [12]:
df_credit_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,585000.0,Unaccompanied,Working,Incomplete higher,Single / not married,House / apartment,0.019689,-8676,-813,-4163.0,-1363,NaN,1,1,1,1,0,0,Core staff,1.0,2,2,FRIDAY,17,0,0,0,1,1,0,Trade: type 2,NaN,0.699373,0.171468,0.0619,0.0302,0.9762,0.6736,0.0055,0.0,0.1034,0.1667,0.0417,0.0000,0.0504,0.0507,0.0,0.0,0.0630,0.0313,0.9762,0.6864,0.0055,0.0,0.1034,0.1667,0.0417,0.0000,0.0551,0.0528,0.0,0.0,0.0625,0.0302,0.9762,0.6780,0.0055,0.0,0.1034,0.1667,0.0417,0.0000,0.0513,0.0516,0.0,0.0,reg oper account,block of flats,0.0399,Block,No,0.0,0.0,0.0,0.0,-589.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,324000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.025164,-13583,-223,-3554.0,-3287,NaN,1,1,0,1,0,0,High skill tech staff,2.0,2,2,MONDAY,11,0,0,0,0,0,0,Business Entity Type 3,0.541385,0.199651,0.768808,0.0227,0.0566,0.9806,0.7348,0.0161,0.0,0.1034,0.0417,0.0833,0.0133,0.0185,0.0184,0.0,0.0,0.0231,0.0587,0.9806,0.7452,0.0162,0.0,0.1034,0.0417,0.0833,0.0136,0.0202,0.0192,0.0,0.0,0.0229,0.0566,0.9806,0.7383,0.0162,0.0,0.1034,0.0417,0.0833,0.0135,0.0188,0.0187,0.0,0.0,reg oper account,block of flats,0.0158,Block,No,0.0,0.0,0.0,0.0,-1409.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,454500.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-13993,-6202,-7971.0,-4175,9.0,1,1,1,1,0,0,Managers,2.0,2,2,THURSDAY,15,0,0,0,0,0,0,Business Entity Type 1,NaN,0.704880,0.626304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [13]:
x_treino, x_teste, y_treino, y_teste = train_test_split(df_credit_train.drop(columns=["SK_ID_CURR","TARGET"]),
                                                        df_credit_train["TARGET"],
                                                        test_size = 0.3,
                                                        random_state = 4)

In [14]:
var_num = x_treino.select_dtypes(include = np.number).columns.tolist()
passthrough = list(set(x_treino.columns.to_list()) - set(var_num + ['NAME_EDUCATION_TYPE']+ ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE', 'NAME_FAMILY_STATUS', 'OCCUPATION_TYPE']))

In [52]:



modelo = DecisionTreeClassifier(
    #min_samples_leaf=,
    max_depth=5
)

parametros = {
    #'modelo__num_leaves': [5],
    'modelo__max_depth': [5],
    #'modelo__n_estimators': [100],
    #'modelo__learning_rate': [0.05],
}

def treina_modelo(modelo=modelo, parametros=parametros, penalidade_feat_sel="l2"):

    modelo_feat_select = LogisticRegressionCV(penalty=penalidade_feat_sel, solver='liblinear', scoring='roc_auc')


    lista_ordenada = [
        'Lower secondary',
        'Secondary / secondary special', 
        'Incomplete higher',
        'Higher education', 
        'Academic degree', 
    ]

    pipe_education = Pipeline(steps = [
        ('trata_education', OrdinalEncoder(categories = [lista_ordenada]))
    ])

    pipe_flag = Pipeline(steps = [
        ('trata_flag', OneHotEncoder(drop='first'))
    ])

    pipe_median = Pipeline(steps = [
        ('trata_flag', SimpleImputer(strategy='median'))
    ])

    pipe_selecao = ColumnTransformer(transformers = [
        ('pipe_education', pipe_education, ['NAME_EDUCATION_TYPE']),
        ('pipe_flag', pipe_flag, ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE', 'NAME_FAMILY_STATUS', 'OCCUPATION_TYPE']),
        ('pipe_median', pipe_median, var_num),
        #('outras_colunas', 'passthrough', ['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED']),
    ])

    pipe_std = Pipeline(steps = [
        ('pipe_std', StandardScaler())
    ])

    pipe_select_feat = Pipeline(steps = [
        ('pipe_select_feat', SelectFromModel(modelo_feat_select, max_features=10, threshold=0.1))
    ])


    pipeline = Pipeline(steps = [
        ('preproc', pipe_selecao),
        ('pipe_std', pipe_std),
        ('pipe_select_feat', pipe_select_feat),
        ('modelo', modelo)
    ])

    gscv = GridSearchCV(
        estimator = pipeline,
        param_grid = parametros,
        scoring = 'roc_auc',
        refit = True,
        cv = 3
    )

    gscv.fit(x_treino, y_treino)
    y_pred_treino = gscv.predict_proba(x_treino)[:, 1]
    y_pred_teste = gscv.predict_proba(x_teste)[:, 1]

    roc_auc_treino = roc_auc_score(y_treino, y_pred_treino)
    roc_auc_teste = roc_auc_score(y_teste, y_pred_teste)

    best_params = gscv.best_params_

    return roc_auc_treino, roc_auc_teste, best_params


In [53]:

modelo = DecisionTreeClassifier(
    max_depth=5
)

parametros = {
    'modelo__max_depth': [5],
}

roc_auc_treino, roc_auc_teste, best_params = treina_modelo(modelo=modelo, parametros=parametros, penalidade_feat_sel="l1")

resultados = pd.DataFrame({"Modelo":"DecisionTree",
                   "roc_auc_treino":roc_auc_treino,
                   "roc_auc_teste":roc_auc_teste,
                   "parametros": [best_params],
                   })

resultados


TypeError: __init__() got an unexpected keyword argument 'scoring'

In [ ]:

modelo = DecisionTreeClassifier(
    max_depth=5
)

parametros = {
    'modelo__max_depth': [5],
}

roc_auc_treino, roc_auc_teste, best_params = treina_modelo(modelo=modelo, parametros=parametros, penalidade_feat_sel="l1")

resultados = pd.DataFrame({"Modelo":"DecisionTree",
                   "roc_auc_treino":roc_auc_treino,
                   "roc_auc_teste":roc_auc_teste,
                   "parametros": [best_params],
                   })

resultados


In [ ]:

modelo = DecisionTreeClassifier()

parametros = {
    'modelo__min_samples_leaf': [2,5,8,10,15,20,25],
    'modelo__max_depth': [2,5,8,10,15,20,25],
}

roc_auc_treino, roc_auc_teste, best_params = treina_modelo(modelo=modelo, parametros=parametros, penalidade_feat_sel="l1")

resultados = pd.DataFrame({"Modelo":"DecisionTree - Tuned",
                   "roc_auc_treino":roc_auc_treino,
                   "roc_auc_teste":roc_auc_teste,
                   "parametros": [best_params],
                   })

resultados


In [ ]:

modelo = DecisionTreeClassifier()

parametros = {
    'modelo__min_samples_leaf': [2,5,8,10,15,20,25],
    'modelo__max_depth': [2,5,8,10,15,20,25],
}

roc_auc_treino, roc_auc_teste, best_params = treina_modelo(modelo=modelo, parametros=parametros, penalidade_feat_sel="l2")

resultados = pd.DataFrame({"Modelo":"DecisionTree - Tuned",
                   "roc_auc_treino":roc_auc_treino,
                   "roc_auc_teste":roc_auc_teste,
                   "parametros": [best_params],
                   })

resultados
